In [35]:
import pandas as pd, numpy as np
import requests
import json
import time
import googlemaps
import prettyprint
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [36]:
df = pd.read_csv('datasets/df_venues2022.csv')

In [37]:
df = df.loc[:,['District']] # getting only latitude and longitude

In [40]:
districts = df.District.values

In [41]:
mc_donalds = [0,0,3,2,2,2,5,0,1,3,1,3,2,1,0,2,0,4,3,4,1,2,10,0,3,1,4,6,0,2,0,0,0,0,7,2,4,5,1]

In [42]:
burger_king =[0,5,8,4,6,8,17,7,5,9,2,4,4,4,0,4,1,6,10,7,3,3,12,6,6,7,7,12,3,7,2,3,2,0,13,4,11,7,2]

In [43]:
df_burger_mcdonalds = pd.DataFrame(list(zip(districts,burger_king,mc_donalds)),columns = ['District','Burger King','Mc Donalds'])

In [46]:
df_burger_mcdonalds.head()

,District,Burger King,Mc Donalds
0,Adalar,0,0
1,Arnavutköy,5,0
2,Ataşehir,8,3
3,Avcılar,4,2
4,Bağcılar,6,2


### Getting KFC Branches in Istanbul 

In [47]:
url = 'https://kfcturkiye.com/restoranlar'

In [48]:
html_text = requests.get(url).content

In [49]:
soup = BeautifulSoup(html_text,'lxml')

In [50]:
subeler = soup.find_all('div', attrs={'class':"text-content"} )

In [51]:
sube_yeri = []
sube_address = []
for i in subeler:
    sube_yeri.append(i.h3.text)
    sube_address.append(i.p.text)

In [52]:
liste = list(zip(sube_yeri,sube_address))

In [53]:
df_kfc = pd.DataFrame(liste,columns = ['Sube_Adi','Address'])

In [54]:
df_kfc= df_kfc[df_kfc['Sube_Adi'].str.contains('İstanbul')]

In [55]:
df_ilce = pd.DataFrame(df_kfc['Address'].str.split(',',expand=True)[1]).rename(columns = {1:'ilce'}) # creating new column using split method

In [56]:
df_kfc = pd.concat([df_kfc,df_ilce],axis=1)

In [57]:
df_kfc=df_kfc.reset_index(drop=True)

In [61]:
df_kfc.head()

,ilce,KFC
0,Ataşehir,6
1,Avcılar,2
2,Bahçelievler,3
3,Bakırköy,6
4,Bayrampasa,1


#### Data Cleaning and Preparation for KFC

In [62]:
df_kfc.loc[15,'ilce'] = 'Bayrampasa'
df_kfc.loc[43,'ilce'] = 'Kağıthane'
df_kfc.loc[76,'ilce']= 'Kadıköy'
df_kfc.loc[87,'ilce'] = 'Eyüp'
df_kfc.loc[31,'ilce'] = 'Bakırköy'
df_kfc.loc[79,'ilce'] = 'Sultangazi'
df_kfc.loc[28,'ilce'] = 'Beylikdüzü'
df_kfc.loc[30,'ilce'] = 'Eyüp'
df_kfc.ilce = df_kfc.ilce.apply(lambda x: x.strip(' '))
df_kfc.ilce = df_kfc.ilce.apply(lambda x : x.replace('Eyüpsultan','Eyüp'))
df_kfc= df_kfc.groupby('ilce')['Sube_Adi'].count().to_frame().reset_index().rename(columns = {'Sube_Adi':'KFC'})

KeyError: 'Column not found: Sube_Adi'

In [63]:
df_new = pd.merge(df_burger_mcdonalds,df_kfc,how='left',left_on='District',right_on='ilce')
df_new.KFC = df_new.KFC.fillna(0)
df_new = df_new.drop('ilce',axis=1)
df_new.KFC= df_new.KFC.astype('int')

In [65]:
df_new.to_csv('competitors.csv',index=False)